# Importanción de librerías 

In [1]:
import tensorflow as tf
import sklearn
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import librosa
import os

In [2]:
tf.test.is_gpu_available()

True

# Generación de ficheros

Esta función se encarga de generar un archivo txt de manera que este contenga 2 elementos por fila ('name_file','etiqueta') de esta manera permitirá poder entrenar la red.

In [3]:
import numpy as np
import os
def generate_file_data(dir,name):
    directory=dir
    #el nombre de los archivos posee el primer dígito en el nombre de esta forma permitirá etiquetarlos.
    a={'0':'cero','1':'uno','2':'dos','3':'tres','4':'cuatro','5':'cinco','6':'seis','7':'siete','8':'ocho','9':'nueve'}
    da=os.listdir(directory)
    # ordena los archivos
    da.sort()
    file = open(dir+name+'.txt',"w")
    for filename in da:
        if '.wav' in filename:
            file.write(filename+','+a[filename[0]]+'\n')
    file.close() 
    # genera el fichero
    with open(directory+'/'+name+'.txt') as f:
        read_data = f.read()
        f.closed
    read_data=read_data.split('\n')
    read_data=read_data[0:len(read_data)-1]
    return read_data

# Encoding words with One Hot Encoding
En esta sección se usará one hot encoding para representar las palabras

In [4]:
from sklearn.preprocessing import OneHotEncoder
vocabulary_words=np.array(['cero','uno','dos','tres','cuatro','cinco','seis','siete','ocho','nueve'])

In [5]:
onehot_encoder = OneHotEncoder(handle_unknown='ignore',categories='auto')

In [6]:
onehot_encoder.fit(X=vocabulary_words.reshape(-1,1))

OneHotEncoder(categorical_features=None, categories='auto',
       dtype=<class 'numpy.float64'>, handle_unknown='ignore',
       n_values=None, sparse=True)

In [7]:
onehot_encoder.categories_

[array(['cero', 'cinco', 'cuatro', 'dos', 'nueve', 'ocho', 'seis', 'siete',
        'tres', 'uno'], dtype='<U6')]

In [8]:
v=onehot_encoder.transform(vocabulary_words.reshape(-1,1)).toarray()

In [9]:
v

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

In [10]:
onehot_encoder.inverse_transform(v)

array([['cero'],
       ['uno'],
       ['dos'],
       ['tres'],
       ['cuatro'],
       ['cinco'],
       ['seis'],
       ['siete'],
       ['ocho'],
       ['nueve']], dtype='<U6')

### Se muestran las funciones para codificar y decodificar

In [11]:
def encode(x):# tomará un array de string y lo transformada a encode
    return onehot_encoder.transform(x.reshape(-1,1)).toarray()
def decode(x):
    return onehot_encoder.inverse_transform(x)

In [12]:
a=encode(np.array(['uno','dos']))

In [13]:
a

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]])

In [14]:
decode(a)

array([['uno'],
       ['dos']], dtype='<U6')

# MFCC 


In [26]:
def mfcc_features(DIR,list_dir,mean,delta):
    mfccd_audios=[]
    for dir in list_dir:
        wave, sr = librosa.load(DIR+dir, mono=True,sr=16000)
        features= librosa.feature.mfcc(wave, sr,n_mfcc=13)
        if delta == True:
            features=librosa.feature.delta(features)
        if mean == True:
            features=sklearn.preprocessing.scale(features)
        try:
            features=np.pad(features,((0,0),(0,110-len(features[0]))),mode='constant', constant_values=0)
        except OSError as err:
            print(dir)
        mfccd_audios.append(features)
        #mfccd_audios.append(np.array([features[0],featuresdelta[0]]))
    mfccd_audios=np.array(mfccd_audios)
    return mfccd_audios

In [17]:
def prepare_data(dir,name,mean,delta):
    file = open(dir+name)
    f=file.read()
    file.close()
    f=f.split('\n')
    f=f[0:len(f)-1]
    labels=[]
    names_audios=[]
    for i in f:
        j=i.split(',')
        names_audios.append(j[0])
        labels.append(j[1])
    labels=np.array(labels)
    onehot= encode(labels)
    mfcc=mfcc_features(dir,names_audios,mean,delta)
    print(name+' OK')
    return mfcc,onehot



In [27]:
class dataset:
    def __init__(self,data):
        self.i=0#para el shuffle
        self.data_dir=data
        self.shuffle=None
        self.dir_training=data+'/training/'
        self.dir_test=data+'/test/'
        self.training_set=None
        self.test_set=None
    def split_dataset(self):
        if os.path.exists(self.dir_training+'training.txt')==False:
            generate_file_data(self.dir_training,name='training')
        if os.path.exists(self.dir_training+'test.txt')==False:
            generate_file_data(self.dir_test,name='test')
        print('loadfiles')
    def prepare(self,mean,delta):
        self.training_set=prepare_data(self.dir_training,'training.txt',mean,delta)
        self.test_set=prepare_data(self.dir_test,'test.txt',mean,delta)

    

In [28]:
d=dataset('data')
d.split_dataset()

loadfiles


In [29]:
d.prepare(mean=False,delta=True)

/home/visoc/anaconda3/envs/tf/lib/python3.6/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


training.txt OK


/home/visoc/anaconda3/envs/tf/lib/python3.6/site-packages/scipy/signal/_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


test.txt OK


# Red neuronal 

En esta sección se definen los distintos modelos a aplicar en este seminario

In [31]:
import sklearn
import tensorflow as tf
import time 
import matplotlib.pyplot as plt
import json
import os

In [32]:
# Se dividen los conjuntos
trainX, trainY = d.training_set[0],d.training_set[1]
testX, testY = d.test_set[0],d.test_set[1]
trainX=np.matrix.transpose(trainX,[0,2,1])
testX=np.matrix.transpose(testX,[0,2,1])

##### A continuación mostramos un elemento del training y test

In [33]:
trainX[0]

array([[ 8.62005689, -6.46370755,  5.88644224, ...,  1.03480394,
        -0.60770764,  0.30691467],
       [ 8.62005689, -6.46370755,  5.88644224, ...,  1.03480394,
        -0.60770764,  0.30691467],
       [ 8.62005689, -6.46370755,  5.88644224, ...,  1.03480394,
        -0.60770764,  0.30691467],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [34]:
trainX.shape

(240, 110, 13)

In [35]:
trainX[0]

array([[ 8.62005689, -6.46370755,  5.88644224, ...,  1.03480394,
        -0.60770764,  0.30691467],
       [ 8.62005689, -6.46370755,  5.88644224, ...,  1.03480394,
        -0.60770764,  0.30691467],
       [ 8.62005689, -6.46370755,  5.88644224, ...,  1.03480394,
        -0.60770764,  0.30691467],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [36]:
testX[0]

array([[ 7.61172524,  3.15274442, -0.62109424, ...,  0.46247704,
         0.95521544,  0.47579891],
       [ 7.61172524,  3.15274442, -0.62109424, ...,  0.46247704,
         0.95521544,  0.47579891],
       [ 7.61172524,  3.15274442, -0.62109424, ...,  0.46247704,
         0.95521544,  0.47579891],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

## Modelos de prueba
A continuación presentamos 4 modelos que serán entrenados para el reconocimiento de voz.

In [37]:
def RNN_simple(name,n_units=128,time_steps=110,n_inputs=13,batch_size=10,n_epochs=400):
    n_class=10
    dir='Models/Model_'
    model=tf.keras.Sequential()
    model.add(tf.keras.layers.SimpleRNN(128, input_shape=(time_steps,n_inputs)))
    model.add(tf.layers.Dense(n_class, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    timei=time.time()
    history=model.fit(trainX,trainY,batch_size=batch_size,epochs=n_epochs,validation_data=[testX,testY])
    timef=time.time()
    if os.path.exists(dir+name) == False:
        os.mkdir(dir+name)
    model.save(dir+name+'/'+name)
    t=timef-timei
    file = open(dir+name+'/'+name+'_time.txt',"w")
    file.write(str(t))
    file.close()
    return history

In [38]:
def LSTM_simple(name,n_units=128,time_steps=110,n_inputs=13,batch_size=10,n_epochs=400):
    n_class=10
    dir='Models/Model_'
    model=tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(n_units, input_shape=(time_steps,n_inputs)))
    model.add(tf.layers.Dense(n_class, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    timei=time.time()
    history=model.fit(trainX,trainY,batch_size=batch_size,epochs=n_epochs,validation_data=[testX,testY])
    timef=time.time()
    if os.path.exists(dir+name) == False:
        os.mkdir(dir+name)
    model.save(dir+name+'/'+name)
    t=timef-timei
    file = open(dir+name+'/'+name+'_time.txt',"w")
    file.write(str(t))
    file.close()
    return history

In [39]:
def LSTM_with_Dropout(name,n_units=128,time_steps=110,n_inputs=13,batch_size=10,n_epochs=400,dropout=0.5):
    n_class=10
    dir='Models/Model_'
    model=tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(n_units, input_shape=(time_steps,n_inputs)))
    model.add(tf.layers.Dense(n_class, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    timei=time.time()
    history=model.fit(trainX,trainY,batch_size=batch_size,epochs=n_epochs,validation_data=[testX,testY])
    timef=time.time()
    if os.path.exists(dir+name) == False:
        os.mkdir(dir+name)
    model.save(dir+name+'/'+name)
    t=timef-timei
    file = open(dir+name+'/'+name+'_time.txt',"w")
    file.write(str(t))
    file.close()
    return history

In [40]:
def LSTM_with_2layers_1D(name,dropout,n_units=13,time_steps=110,n_inputs=20,batch_size=10,n_epochs=400):
    n_class=10
    dir='Models/Model_'
    model=tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(n_units, input_shape=(time_steps,n_inputs),return_sequences=True))
    #model.add(tf.keras.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None))
    model.add(tf.keras.layers.LSTM(n_units, input_shape=(time_steps,n_inputs)))
    model.add(tf.keras.layers.Dropout(dropout))
    model.add(tf.layers.Dense(n_class, activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    timei=time.time()
    history=model.fit(trainX,trainY,batch_size=batch_size,epochs=n_epochs,validation_data=[testX,testY])
    timef=time.time()
    if os.path.exists(dir+name) == False:
        os.mkdir(dir+name)
    model.save(dir+name+'/'+name)
    t=timef-timei
    file = open(dir+name+'/'+name+'_time.txt',"w")
    file.write(str(t))
    file.close()
    return history

## Plotting and saving modelo
Se presentará una función que graficará los resultados y guardará el modelo y sus datos de entrenamiento

In [41]:
def plot_and_saving(history,name):
    dir='Models/Model_'
    # Plot training & validation accuracy values
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('Precision del Modelo')
    plt.ylabel('Precision')
    plt.xlabel('Epoch')
    plt.legend(['Training', 'Test'], loc='upper left')
    plt.savefig(dir+name+'/prec')
    plt.show()
    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Perdida del modelo')
    plt.ylabel('Perdida')
    plt.xlabel('Epoch')
    plt.legend(['Training', 'Test'], loc='upper left')
    plt.savefig(dir+name+'/cost')
    plt.show()
    json.dump(history.history, open('Models/Model_'+name+'/'+name+'_history', 'w'))


# Entrenamiento 
Entrenaremos distintas con los modelos definidos

In [ ]:
RNN=RNN_simple('RNNsimple13mfcc')

Train on 240 samples, validate on 110 samples
Epoch 1/400
240/240 [==============================] - 2s 10ms/step - loss: 2.3527 - acc: 0.1042 - val_loss: 2.2825 - val_acc: 0.1273
Epoch 2/400
240/240 [==============================] - 2s 9ms/step - loss: 2.3536 - acc: 0.1208 - val_loss: 2.4042 - val_acc: 0.1000
Epoch 3/400
240/240 [==============================] - 2s 7ms/step - loss: 2.3973 - acc: 0.0833 - val_loss: 2.3855 - val_acc: 0.0455
Epoch 4/400
240/240 [==============================] - 2s 7ms/step - loss: 2.3632 - acc: 0.1292 - val_loss: 2.3634 - val_acc: 0.1364
Epoch 5/400
240/240 [==============================] - 2s 7ms/step - loss: 2.3862 - acc: 0.1000 - val_loss: 2.3028 - val_acc: 0.1455
Epoch 6/400
240/240 [==============================] - 2s 8ms/step - loss: 2.3487 - acc: 0.0958 - val_loss: 2.3192 - val_acc: 0.1364
Epoch 7/400
240/240 [==============================] - 2s 7ms/step - loss: 2.3397 - acc: 0.1083 - val_loss: 2.3377 - val_acc: 0.1182
Epoch 8/400
240/240 [=

Epoch 62/400
240/240 [==============================] - 3s 12ms/step - loss: 2.2634 - acc: 0.1208 - val_loss: 2.4467 - val_acc: 0.0818
Epoch 63/400
240/240 [==============================] - 1s 5ms/step - loss: 2.2627 - acc: 0.1542 - val_loss: 2.4406 - val_acc: 0.0818
Epoch 64/400
240/240 [==============================] - 1s 5ms/step - loss: 2.2588 - acc: 0.1458 - val_loss: 2.4495 - val_acc: 0.0818
Epoch 65/400
240/240 [==============================] - 1s 5ms/step - loss: 2.2650 - acc: 0.1333 - val_loss: 2.4445 - val_acc: 0.0545
Epoch 66/400
240/240 [==============================] - 2s 6ms/step - loss: 2.2685 - acc: 0.1583 - val_loss: 2.4419 - val_acc: 0.0727
Epoch 67/400
240/240 [==============================] - 2s 9ms/step - loss: 2.2735 - acc: 0.1458 - val_loss: 2.4658 - val_acc: 0.1000
Epoch 68/400
240/240 [==============================] - 2s 7ms/step - loss: 2.2695 - acc: 0.1333 - val_loss: 2.4404 - val_acc: 0.0545
Epoch 69/400
240/240 [==============================] - 2s 6m

Epoch 123/400
240/240 [==============================] - 1s 6ms/step - loss: 2.2383 - acc: 0.1458 - val_loss: 2.6214 - val_acc: 0.0818
Epoch 124/400
240/240 [==============================] - 1s 6ms/step - loss: 2.2321 - acc: 0.1292 - val_loss: 2.5919 - val_acc: 0.0636
Epoch 125/400
240/240 [==============================] - 2s 6ms/step - loss: 2.2435 - acc: 0.1625 - val_loss: 2.6372 - val_acc: 0.0636
Epoch 126/400
240/240 [==============================] - 1s 6ms/step - loss: 2.2223 - acc: 0.1542 - val_loss: 2.5842 - val_acc: 0.1000
Epoch 127/400
240/240 [==============================] - 1s 5ms/step - loss: 2.2189 - acc: 0.1500 - val_loss: 2.6011 - val_acc: 0.0818
Epoch 128/400
240/240 [==============================] - 1s 6ms/step - loss: 2.2146 - acc: 0.1417 - val_loss: 2.6520 - val_acc: 0.0545
Epoch 129/400
240/240 [==============================] - 1s 6ms/step - loss: 2.2199 - acc: 0.1292 - val_loss: 2.6364 - val_acc: 0.0727
Epoch 130/400
240/240 [==============================] 

240/240 [==============================] - 2s 7ms/step - loss: 2.2133 - acc: 0.0792 - val_loss: 2.7147 - val_acc: 0.0545
Epoch 184/400
240/240 [==============================] - 2s 7ms/step - loss: 2.2224 - acc: 0.1333 - val_loss: 2.6709 - val_acc: 0.0455
Epoch 185/400
240/240 [==============================] - 1s 5ms/step - loss: 2.2053 - acc: 0.1292 - val_loss: 2.7057 - val_acc: 0.0636
Epoch 186/400
240/240 [==============================] - 1s 5ms/step - loss: 2.2110 - acc: 0.1500 - val_loss: 2.6493 - val_acc: 0.0636
Epoch 187/400
240/240 [==============================] - 2s 6ms/step - loss: 2.2071 - acc: 0.1417 - val_loss: 2.6640 - val_acc: 0.0727
Epoch 188/400
240/240 [==============================] - 2s 8ms/step - loss: 2.2121 - acc: 0.1292 - val_loss: 2.6424 - val_acc: 0.0818
Epoch 189/400
240/240 [==============================] - 1s 5ms/step - loss: 2.1990 - acc: 0.1292 - val_loss: 2.6379 - val_acc: 0.0273
Epoch 190/400
240/240 [==============================] - 1s 5ms/step 

Epoch 244/400
240/240 [==============================] - 2s 9ms/step - loss: 2.2191 - acc: 0.1583 - val_loss: 2.5620 - val_acc: 0.0636
Epoch 245/400
240/240 [==============================] - 2s 9ms/step - loss: 2.2168 - acc: 0.1542 - val_loss: 2.5832 - val_acc: 0.0636
Epoch 246/400
240/240 [==============================] - 2s 9ms/step - loss: 2.2160 - acc: 0.1792 - val_loss: 2.5692 - val_acc: 0.0727
Epoch 247/400
240/240 [==============================] - 2s 9ms/step - loss: 2.2184 - acc: 0.1250 - val_loss: 2.5942 - val_acc: 0.0727
Epoch 248/400
240/240 [==============================] - 2s 9ms/step - loss: 2.2185 - acc: 0.1458 - val_loss: 2.5561 - val_acc: 0.0727
Epoch 249/400
240/240 [==============================] - 2s 9ms/step - loss: 2.2010 - acc: 0.1542 - val_loss: 2.5708 - val_acc: 0.0727
Epoch 250/400
240/240 [==============================] - 2s 7ms/step - loss: 2.2219 - acc: 0.1542 - val_loss: 2.6088 - val_acc: 0.0818
Epoch 251/400
240/240 [==============================] 

190/240 [======================>.......] - ETA: 0s - loss: 2.2094 - acc: 0.1789

In [ ]:
plot_and_saving(RNN,'RNNsimple13mfcc')

In [ ]:
LSTM_SIMPLE=LSTM_simple(name='LSTM_13mfcc')

In [ ]:
plot_and_saving(LSTM_SIMPLE,name='LSTM_13mfcc')

In [ ]:
LSTMD5=LSTM_with_Dropout('LSTM_Dropout5_mfcc13',dropout=0.5)

In [ ]:
plot_and_saving(LSTMD5,name='LSTM_Dropout5_mfcc13')

In [ ]:
LSTMD8=LSTM_with_Dropout('LSTM_Dropout8_mfcc13',dropout=0.8)

In [ ]:
plot_and_saving(LSTMD8,name='LSTM_Dropout8_mfcc13')

In [ ]:
LSTM2_1=LSTM_with_2layers_1D('LSTM_2Layer1DDelta8',dropout=0.8)

In [ ]:
plot_and_saving(LSTM2_1,name='LSTM_2Layer1D_13mfcc)

In [ ]:
LSTM2_1iter900=LSTM_with_2layers_1D('LSTM_2Layer1D_13mfcc_Delta8',dropout=0.8)

In [ ]:
LSTM2_1iter900=LSTM_with_2layers_1D('LSTM_2Layer1D_13mfcc_Delta8',dropout=0.8)
plot_and_saving(LSTM2_1,name='LSTM_2Layer1D_13mfcc)

# Load Model 

In [35]:
import tensorflow as tf
from keras.utils import plot_model

Using TensorFlow backend.


In [37]:
def load_model(dir):
    m=tf.keras.models.load_model(dir)
    return m

# Recording audios and predict them

In [1]:
import sounddevice as sd
import ipywidgets as widgets
from IPython.display import display

In [3]:
button1 = widgets.Button(description="Record",)
button2 = widgets.Button(description='Test')
display(button)

def on_button_clicked(b):
    print("Button clicked.")

button1.on_click(on_button_clicked)
button2.on_click(on_button_clicked)


Button(description='Record', style=ButtonStyle())

In [ ]:
import pyaudio
import wave
 
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 48000#44100
CHUNK = 1024
RECORD_SECONDS = 2
WAVE_OUTPUT_FILENAME = "file.wav"
 
audio = pyaudio.PyAudio()
 
# start Recording
stream = audio.open(format=FORMAT, channels=CHANNELS,
                rate=RATE, input=True,
                frames_per_buffer=CHUNK)
print ("recording...")
frames = []
 
for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
    data = stream.read(CHUNK)
    frames.append(data)
print ("finished recording")
 
 
# stop Recording
stream.stop_stream()
stream.close()
audio.terminate()
 
waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
waveFile.setnchannels(CHANNELS)
waveFile.setsampwidth(audio.get_sample_size(FORMAT))
waveFile.setframerate(RATE)
waveFile.writeframes(b''.join(frames))
waveFile.close()

recording...


In [178]:
wave

<module 'wave' from '/home/visoc/anaconda3/envs/tf/lib/python3.6/wave.py'>

In [186]:
import librosa
wave, sr = librosa.load('file.wav', mono=True)
features= librosa.feature.mfcc(wave, sr,n_mfcc=20)

In [180]:
wave

array([-0.00155109, -0.00390198, -0.00120344, ...,  0.00196864,
        0.00147835,  0.        ], dtype=float32)

In [187]:
#features = sklearn.preprocessing.scale(features, axis=1)
features=np.pad(features,((0,0),(0,160-len(features[0]))),mode='constant', constant_values=0)
f=np.matrix.transpose(np.array([features]),[0,2,1])

In [188]:
label=model.predict_classes(f)

<bound method Sequential.get_config of <tensorflow.python.keras.engine.sequential.Sequential object at 0x7f4076c35128>>


In [189]:
decode(np.eye(10)[label])

array([['cuatro']], dtype='<U6')

In [35]:
label=class_to_integer_encoded(label[0]-1)

In [36]:
label

array([[6]])

In [37]:
label_encoder.inverse_transform(label)

array(['seis'], dtype='<U6')